In [18]:
import sqlite3
import json

# Connect to a SQLite database in memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create the original table
cursor.execute('''
    CREATE TABLE original_table (
        name TEXT,
        relations TEXT
    )
''')

# Insert data into the original table
# For simplicity, we use a small dataset
data = [
    ('Alice', json.dumps(['Bob', 'Charlie', 'David'])),
    ('Bob', json.dumps(['Alice'])),
    ('Charlie', json.dumps(['Alice']))
]

cursor.executemany('INSERT INTO original_table (name, relations) VALUES (?, ?)', data)

# print the data from the original table to verify
cursor.execute('SELECT * FROM original_table')
original_data = cursor.fetchall()
for row in original_data:
    print(row)

# Create the new 'relations' table
cursor.execute('''
    CREATE TABLE relations (
        r1 TEXT,
        r2 TEXT
    )
''')

# Insert data into the 'relations' table by extracting pairs from the original table
cursor.execute('''
    INSERT INTO relations (r1, r2)
    SELECT original_table.name, json_each.value
    FROM original_table
    CROSS JOIN json_each(original_table.relations)
''')
# WHERE o.name <> json_each.value

# Fetch and print the data from the 'relations' table to verify
cursor.execute('SELECT * FROM relations')
relations_data = cursor.fetchall()

relations_data



('Alice', '["Bob", "Charlie", "David"]')
('Bob', '["Alice"]')
('Charlie', '["Alice"]')


[('Alice', 'Bob'),
 ('Alice', 'Charlie'),
 ('Alice', 'David'),
 ('Bob', 'Alice'),
 ('Charlie', 'Alice')]

In [23]:
# select * from original_table
query = """
SELECT original_table.name, json_each.value
FROM original_table
CROSS JOIN json_each(original_table.relations);

"""
cursor.execute(query).fetchall()

[('Alice', 'Bob'),
 ('Alice', 'Charlie'),
 ('Alice', 'David'),
 ('Bob', 'Alice'),
 ('Charlie', 'Alice')]